<br>
<div style="font-size:83px; font-weight:bold; text-align: center;">Learn You a Neural Net! </div>
<br>

`whoami`  

`stu`  
Machine Learning Engineer @Opendoor  
@mstewart141  

# Goal: Puzzle through the mechanics of a fully connected neural net on our way to implementing one in a modular/reusable style

Per Wikipedia:
> An ANN is based on a collection of connected units or **neurons**... the output of each artificial neuron is calculated by a **non-linear function** of the **sum of its inputs**... Artificial neurons and edges typically have a **weight** that adjusts as learning proceeds. The weight increases or decreases the strength of the signal at a connection... Typically, neurons are organized in **layers**... Signals travel from the first (input), to the last (output) layer (...)

![title](ann2.jpeg)
[(_source_)](https://towardsdatascience.com/multi-layer-neural-networks-with-sigmoid-function-deep-learning-for-rookies-2-bf464f09eb7f)

## Artificial neurons and edges typically have a **weight** that adjusts as learning proceeds

### Ok but how do we make learning *proceed*?

### Almost always, we rely on a technique in the gradient descent family of algorithms!

Per Wikipedia:
> Gradient descent is a first-order iterative optimization algorithm for finding the __minimum__ of a function. To find a local minimum of a function using gradient descent, one takes steps proportional to the negative of the gradient (or of the approximate gradient) of the function at the current point

# First, some data initialization.

In [1]:
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import enum
from typing import List, Callable

import numpy as np
from sklearn import base, linear_model
from sympy import var, plot, diff, solve, exp, tanh, functions, stats
from toolz import pipe

Tensor = np.array

In [3]:
X = np.array([[0, 0, 1],  # XOR problem
              [0, 1, 1],
              [1, 0, 1],
              [0, 1, 0],
              [1, 0, 0],
              [1, 1, 1]])
Y = np.array([[0, 1, 1, 1, 1, 0]]).T

sig = lambda z: 1 / (1 + np.exp(-z))
sig_prime = lambda z: z * (1 - z)  # three lines of calculus later

### _XOR_  is a bit wonky.

### Linear model? No dice.

In [4]:
linear_model.LogisticRegression(C=1e18).fit(X, Y.ravel()).predict(X).reshape(-1, 1)

array([[0],
       [0],
       [0],
       [1],
       [1],
       [0]])

# What would a NN / MLP look like in a for loop?

## _(a.k.a. gotta start somewhere)_

### Aside: we'll use `sigmoid` as a non-linear activation because it is widely known:

# $$\begin{equation} \sigma(z) = 1\space /\space {(1+e^{-z})} \end{equation}$$

In [6]:
betas0_1 = np.random.random(size=(3, 4)) - 0.5
betas1_2 = np.random.random(size=(4, 1)) - 0.5

lr = 16
Z = np.matmul

for i in range(30000):
    # forward
    layer0 = X
    layer1 = sig(Z(layer0, betas0_1))
    layer2 = sig(Z(layer1, betas1_2))  # hypothesis for NN
    
    # backward 
    layer2_loss = layer2 - Y
    layer2_delta = layer2_loss * sig_prime(layer2)
    grad1_2 = layer1.T @ layer2_delta  # find our gradients
    
    layer1_loss = layer2_delta @ betas1_2.T
    layer1_delta = layer1_loss * sig_prime(layer1)
    grad0_1 = layer0.T @ layer1_delta    
    
    betas0_1 = betas0_1 - lr * grad0_1  # update iteratively
    betas1_2 = betas1_2 - lr * grad1_2

else:
    print(np.round(layer2, 4))

[[ 0.0016]
 [ 0.9989]
 [ 0.9985]
 [ 1.    ]
 [ 0.9999]
 [ 0.0014]]


# Why is this so? Chain rule!

# $Cost = C(\sigma(Z(X W)))$

## $where \space\space C(\hat{y}, y) = \frac{1}{2}(\hat{y} - y)^2$

# $\begin{split}C'(W) &= C'(\sigma) \cdot \sigma'(Z) \cdot Z'(W) \\
      &= (\hat{y} -y) \cdot \sigma'(Z) \cdot X\end{split}$
      
(_latex [source](https://ml-cheatsheet.readthedocs.io/en/latest/backpropagation.html)_)

# Ok, your turn!

# Now, what could go wrong with the above plan?

# Ok! But, how can we generalize this??

In [9]:
class LayerKind(enum.Enum):
    SIGMOID = (sig, sig_prime)  # activation functions
    TANH = (np.tanh, lambda z: 1 - z ** 2)
    def __init__(self, fn: Callable, fn_prime: Callable):
        self.fn = fn
        self.fn_prime = fn_prime

K = LayerKind

In [14]:
class Layer:
    def __init__(self, kind: LayerKind, shape: tuple):
        self.fn = kind.fn
        self.fn_prime = kind.fn_prime
        self.betas = np.random.random(shape) - 0.5
    def forward(self, layer_input: Tensor) -> Tensor:
        self.input = layer_input
        self.activated = self.fn(self.input @ self.betas)
        return self.activated
    def backward(self, this_layer_loss: Tensor) -> Tensor:
        '''Backward propagate gradient info for a mXk layer where prior layer was mXn and next layer is mXj.

        So, layer input is mXn, `this_layer_loss` is mXk, and betas are nXk. Grad best be nXk like our errors!

        self.activated: mXk
        this_layer_loss: mXk
        error_signal: mXk
        self.betas: nXk
        grad: nXk
        self.input: mXn
        prev_layer_loss: mXn
        '''
        error_signal = this_layer_loss * self.fn_prime(self.activated)
        grad = self.input.T @ error_signal
        self.betas = self.betas - 0.4 * grad
        prev_layer_loss = error_signal @ self.betas.T
        return prev_layer_loss

In [17]:
class NN(base.BaseEstimator):
    """NN uses `Layer` and basically does nothing."""
    
    def __init__(self, layers: List[Layer]):
        self.layers = layers
    
    def _forward(self, X: Tensor) -> Tensor:
        return pipe(X, *(layer.forward for layer in self.layers))
    
    def _backward(self, last_layer_loss: Tensor) -> None:
        pipe(last_layer_loss, *(layer.backward for layer in reversed(self.layers)))
    
    def fit(self, X: Tensor, Y: Tensor) -> 'NN':
        for _ in range(60000):
            hypothesis = self._forward(X)
            self._backward(hypothesis - Y)
        return self
    
    predict_proba = _forward
    
    def predict(self, X: Tensor) -> Tensor:
        return pipe(X, self.predict_proba, np.round, lambda arr: arr.astype(int))

In [23]:
y_hat = NN(layers=[
    Layer(kind=K.SIGMOID, shape=(3, 4)),
    Layer(kind=K.SIGMOID, shape=(4, 1)),
]).fit(X, Y).predict(X)

In [24]:
all(y_hat == Y)

True

# Ok, your turn!

# Can we implement a better stopping criterion?

<br>
<div style="font-size:80px; font-weight:bold; text-align: center;"> Questions? </div>
<br>
  
  
_@[mstewart141](https://twitter.com/mstewart141) [twitter, github, linkedin]_